In [37]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

In [38]:
def create_bar_plot(df: pd.DataFrame, categorical_column: str, overall_avg: pd.Series, response_dict: dict):
    plt.figure(figsize=(12, 6))
    
    # Calculate means for each score by category
    math_means = df.groupby(categorical_column)['math score'].mean()
    reading_means = df.groupby(categorical_column)['reading score'].mean()
    writing_means = df.groupby(categorical_column)['writing score'].mean()
    
    # Set bar positions
    x = range(len(df[categorical_column].unique()))
    width = 0.25
    
    # Create bars
    plt.bar([i - width for i in x], math_means, width, label='Math')
    plt.bar(x, reading_means, width, label='Reading')
    plt.bar([i + width for i in x], writing_means, width, label='Writing')
    
    # Add overall average points
    plt.plot([i - width for i in x], [overall_avg['math score']] * len(x), 'ro', label='Overall Math')
    plt.plot(x, [overall_avg['reading score']] * len(x), 'go', label='Overall Reading')
    plt.plot([i + width for i in x], [overall_avg['writing score']] * len(x), 'bo', label='Overall Writing')

    # Finalize plot
    plt.title(f'Average Scores by {categorical_column}')  
    plt.xticks(x, df[categorical_column].unique(), rotation=45)
    plt.legend()
    plt.tight_layout()
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    plt.close()
    return image

# response_dict = {}

# # Calculate overall average scores
# score_columns = ['math score', 'reading score', 'writing score']
# categorical_columns = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
# overall_avg = df[score_columns].mean()

# create_bar_plot(df, 'gender', overall_avg, response_dict)


In [39]:
def process_overall_avg(overall_avg: pd.Series, response_dict: dict):
    # Create bar plot for overall averages
    plt.figure(figsize=(10, 6))
    overall_avg.plot(kind='bar')
    plt.title('Overall Average Scores by Subject')
    plt.ylabel('Average Score')
    plt.xticks(rotation=0)
    plt.tight_layout()

    # save the plot to a buffer and convert to base64
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    
    response = {'data': overall_avg.round(2).to_dict(), 'image': image}
    plt.close()
    return response

def process_file(df: pd.DataFrame):
    score_columns = ['math score', 'reading score', 'writing score']
    categorical_columns = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
    overall_avg = df[score_columns].mean()

    response_dict = {}
    response_dict['overall'] = process_overall_avg(overall_avg, response_dict)
    response_dict['columns'] = []
    for col in categorical_columns:
        response_dict['columns'].append({col: {'data': df.groupby(col)[score_columns].mean().round(2).to_dict(), 'image': create_bar_plot(df, col, overall_avg, response_dict)}})
    return response_dict


In [40]:
file_path = "StudentsPerformance.csv"

df = pd.read_csv(file_path)
response_dict = process_file(df)

In [41]:
print(response_dict)

{'overall': {'data': {'math score': 66.09, 'reading score': 69.17, 'writing score': 68.05}, 'image': '...'}, 'columns': [{'gender': {'data': {'math score': {'female': 63.63, 'male': 68.73}, 'reading score': {'female': 72.61, 'male': 65.47}, 'writing score': {'female': 72.47, 'male': 63.31}}, 'image': '...'}}, {'race/ethnicity': {'data': {'math score': {'group A': 61.63, 'group B': 63.45, 'group C': 64.46, 'group D': 67.36, 'group E': 73.82}, 'reading score': {'group A': 64.67, 'group B': 67.35, 'group C': 69.1, 'group D': 70.03, 'group E': 73.03}, 'writing score': {'group A': 62.67, 'group B': 65.6, 'group C': 67.83, 'group D': 70.15, 'group E': 71.41}}, 'image': '...'}}, {'parental level of education': {'data': {'math score': {"associate's degree": 67.88, "bachelor's degree": 69.39, 'high school': 62.14, "master's degree": 69.75, 'some college': 67.13, 'some high school': 63.5}, 'reading score': {"associate's degree": 70.93, "bachelor's degree": 73.0, 'high school': 64.7, "master's de

In [42]:
for item in response_dict['columns']:
    print(item)


{'gender': {'data': {'math score': {'female': 63.63, 'male': 68.73}, 'reading score': {'female': 72.61, 'male': 65.47}, 'writing score': {'female': 72.47, 'male': 63.31}}, 'image': '...'}}
{'race/ethnicity': {'data': {'math score': {'group A': 61.63, 'group B': 63.45, 'group C': 64.46, 'group D': 67.36, 'group E': 73.82}, 'reading score': {'group A': 64.67, 'group B': 67.35, 'group C': 69.1, 'group D': 70.03, 'group E': 73.03}, 'writing score': {'group A': 62.67, 'group B': 65.6, 'group C': 67.83, 'group D': 70.15, 'group E': 71.41}}, 'image': '...'}}
{'parental level of education': {'data': {'math score': {"associate's degree": 67.88, "bachelor's degree": 69.39, 'high school': 62.14, "master's degree": 69.75, 'some college': 67.13, 'some high school': 63.5}, 'reading score': {"associate's degree": 70.93, "bachelor's degree": 73.0, 'high school': 64.7, "master's degree": 75.37, 'some college': 69.46, 'some high school': 66.94}, 'writing score': {"associate's degree": 69.9, "bachelor's 